In [2]:
import os

from langchain_upstage import ChatUpstage
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [3]:
# 단계 1: 문서 로드(Load Documents)
# loader = PyMuPDFLoader("20250821_company_463434000.pdf")
loader = PyMuPDFLoader("/Users/hoppure/LLMproject/pdfs/20250825_company_965562000.pdf")

docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 4


In [4]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")



분할된 청크의수: 12


In [5]:
# 단계 3: 임베딩(Embedding) 생성
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# result = embeddings.embed_query("텍스트 내용")
# print(result)

/var/folders/y6/z4qfl0jj0yzfknm6mrgsdvt40000gn/T/ipykernel_1087/3151492755.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [6]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [29]:
loaded_vs = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)


In [30]:
print(f"로딩된 벡터스토어 문서 수: {len(loaded_vs.docstore._dict)}")


로딩된 벡터스토어 문서 수: 21234


In [7]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [8]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("매출액")

[Document(id='14958c79-fe35-4629-8879-65ce28cb7eb7', metadata={'producer': 'crawler-script', 'creator': 'crawler-script', 'creationdate': 'D:20250825000000', 'source': '/Users/hoppure/LLMproject/pdfs/20250825_company_965562000.pdf', 'file_path': '/Users/hoppure/LLMproject/pdfs/20250825_company_965562000.pdf', 'total_pages': 4, 'format': 'PDF 1.5', 'title': '주목해야 할 NDR 후기', 'author': 'IBK투자증권', 'subject': '대양전기공업', 'keywords': '대양전기공업', 'moddate': 'D:20250825000000', 'trapped': '', 'modDate': 'D:20250825000000', 'creationDate': 'D:20250825000000', 'page': 3}, page_content='21,000 \n-19.37 \n-8.57 \n2025.03.11 \n매수 \n29,000 \n-39.13 \n-30.00 \n2025.05.16 \n매수 \n33,000 \n-28.99 \n-15.61 \n2025.08.25 \n매수 \n33,000 \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n   0\n   5,000\n   10,000\n   15,000\n   20,000\n   25,000\n   30,000\n   35,000\n(원)'),
 Document(id='302037f5-f54c-49cc-acbb-122a3f0

In [9]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Answer in Korean.

#Question:
{question}
#Context:
{context}

#Answer:"""
)

In [ ]:
from mlx_lm import load, generate

# 4bit 양자화된 경량 모델 예시(원하는 모델로 교체 가능)
model_id = "InferenceIllusionist/gpt-oss-20b-MLX-4bit"

model, tokenizer = load(model_id)
prompt = "간단히 자기소개하고, 오늘 날씨에 맞는 작업 루틴을 추천해줘."

text = generate(model, tokenizer, prompt=prompt, max_tokens=128, )
print(text)


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

[WARNING] Generating with a model that requires 11229 MB which is close to the maximum recommended size of 12288 MB. This can be slow. See the documentation for possible work-arounds: https://github.com/ml-explore/mlx-lm/tree/main#large-models


In [10]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.

from dotenv import load_dotenv

# .env 파일 로드 (기본: 현재/상위 경로 탐색)
load_dotenv()

chat = ChatUpstage(
    base_url="https://api.upstage.ai/v1",   # 기본값
    model="solar-pro",                      # 또는 "solar-mini"
    temperature=0.3,
    max_tokens=512,
)

messages = [
    SystemMessage(content="당신은 한국어에 능숙한 전문 AI 어시스턴트입니다."),
    HumanMessage(content="이 모델의 강점은 무엇인가요?"),
]

resp = chat.invoke("이 모델의 강점은 무엇인가요?")
print(resp.content)

현재 제가 기반으로 하는 **Solar Pro 2** 모델의 주요 강점은 다음과 같습니다:

### 1. **고성능 하이브리드 아키텍처**  
   - **Thinking Mode**와 **Non-thinking Mode**를 상황에 따라 전환해 효율성과 정확성을 동시에 확보합니다.  
     - *Thinking Mode*: 복잡한 추론, 창의적 문제 해결에 최적화 (예: 수학 증명, 논리적 분석).  
     - *Non-thinking Mode*: 빠른 응답이 필요한 단순 질의에 적합 (예: 사실 기반 QA, 간단한 번역).  

### 2. **다국어 및 한국어 최적화**  
   - 한국어, 일본어 등 아시아 언어에 특화되어 있으며, 문맥 이해와 자연스러운 생성이 뛰어납니다.  
   - 영어, 독일어, 프랑스어 등 주요 언어도 지원 (총 29개 언어).  

### 3. **대규모 파라미터와 지식**  
   - 30.9B 파라미터 규모로 광범위한 지식(2025년 3월 기준)을 보유하며, 전문 분야(의학, 법률, 공학 등)에서도 높은 이해도를 보입니다.  

### 4. **실시간 정보 처리**  
   - **Solar API**와 연동해 최신 정보(주식, 날씨, 뉴스 등)를 반영한 응답이 가능합니다.  

### 5. **윤리적 안전성**  
   - 편향성 감소, 유해성 필터링 등 **AI 윤리 가이드라인**을 준수하며, 민감한 주제에 대해 책임 있는 답변을 제공합니다.  

### 6. **다양한 활용 사례**  
   - 문서 요약, 코드 생성(Python, Java 등), 대화, 데이터 분석 등 **멀티모달 작업**에 적용 가능합니다.  

### 7. **업스테이지의 기술 통합**  
   - 문서 처리(**OCR/KIE**) 및 **Solar Pro 2**를 결합해 비즈니스 워크플로우 자동화(예: 계약서 분석, 영수증 처리)에 강점을 가집니다.  

> 💡 **차별점**: 일반적인 오픈소스 모델보다 정교한 한국어 처리와 하이브리드 

In [12]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | chat
    | StrOutputParser()
)
"""
# 단계 8: 체인(Chain) 생성
chain2 = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
"""

'\n# 단계 8: 체인(Chain) 생성\nchain2 = (\n    {"context": retriever, "question": RunnablePassthrough()}\n    | prompt\n    | llm\n    | StrOutputParser()\n)\n'

In [11]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "어떤 기업의 리포트야?"
response = chain.invoke(question)
print(response)

제공된 컨텍스트에 따르면, 이 리포트는 **대양전기공업**(종목코드: 108380)에 대한 분석 자료입니다.  
- **작성자**: IBK투자증권  
- **제목**: "주목해야 할 NDR 후기"  
- **발행일**: 2025년 8월 25일  
- **주요 내용**: 분기별 예상 실적 추이, 업종별 매출 분석(조선, 방산, 센서 및 기타), 투자 의견(매수) 및 목표가(21,000원~33,000원) 등이 포함되어 있습니다.  

따라서 이 리포트는 **대양전기공업**이라는 기업을 분석한 보고서입니다.


In [18]:
question = "어떤 기업의 리포트야?"
response = chain2.invoke(question)
print(response)

KeyboardInterrupt: 

In [12]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "이 기업의 시가총액은 얼마야?"
response = chain.invoke(question)
print(response)

"""
현대건설(000720) 시가총액
현재 시가총액(보통주 기준): 약 6조 9,597억 원.

상장예정 포함 시가총액: 약 7조 0,174억 원.

참고 지표: 종가 62,500원, 발행주식수(보통주) 111,355,765주, 외국인지분율 24.44% (기준일 2025-08-21).

"""

제공된 컨텍스트에 따르면 대양전기공업의 시가총액은 2,497억 원입니다. 이 정보는 두 개의 문서(Document ID: cf2134ed-e5c4-4f14-965f-e16ecac189df 및 55423344-1b9b-4ae4-a1fe-27d1e64ffdd5)에서 확인할 수 있습니다. 

답변: 대양전기공업의 시가총액은 2,497억 원입니다.


'\n현대건설(000720) 시가총액\n현재 시가총액(보통주 기준): 약 6조 9,597억 원.\n\n상장예정 포함 시가총액: 약 7조 0,174억 원.\n\n참고 지표: 종가 62,500원, 발행주식수(보통주) 111,355,765주, 외국인지분율 24.44% (기준일 2025-08-21).\n\n'

In [1]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "이 기업의 투자의견 알려줘."
response = chain.invoke(question)
print(response)
"""
현대건설(000720) 투자의견
투자의견: Buy(매수) 유지

목표주가(12M): 83,000원으로 상향(기존 51,000원 대비 +62.7%)

상향 근거: 2027년 원전 수주·실적 기여 기대를 반영한 밸류에이션 적용(25년 EPS에 Target P/E 21배, 27년 기준으로는 9배 적용과 동일한 컨셉)

주요 변수: 원전 수주 여부, 원전 인력 캐파 확장, 주택·건축 부문 리스크에 따른 목표주가 변동 가능성

"""

NameError: name 'chain' is not defined

In [13]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "현대건설의 현재주가와 목표주가 알려줘."
response = chain.invoke(question)
print(response)
"""현대건설(000720) 주가 정보
현재주가: 70,900원(기준일 2025-07-18, 리포트 내 표기)

목표주가(12M): 83,000원으로 상향(이전 51,000원에서 +62.7%)

참고
투자의견: Buy(매수) 유지

목표주가 산정: 2025년 EPS에 Target P/E 21배 적용(2027년 EPS 기준으로는 9배 적용과 동일한 컨셉)

첨부 리포트: 하나증권, 2025-07-21자 현대건설(000720) 기업분석/Earnings Review

"""

제공된 컨텍스트는 대양전기공업에 대한 분석 내용을 담고 있으며, 현대건설의 주가 정보는 포함되어 있지 않습니다. 따라서 현대건설의 현재주가와 목표주가에 대해서는 알 수 없습니다. 

대양전기공업의 경우, 컨텍스트에 따르면 2025년 8월 22일 기준 현재가는 26,100원이며, IBK투자증권의 목표주가는 33,000원으로 유지되고 있습니다. 하지만 이는 현대건설과 무관한 정보입니다. 

현대건설의 주가 정보가 필요하시다면 최신 금융 데이터나 증권사 리포트를 참조하시기 바랍니다.


'현대건설(000720) 주가 정보\n현재주가: 70,900원(기준일 2025-07-18, 리포트 내 표기)\n\n목표주가(12M): 83,000원으로 상향(이전 51,000원에서 +62.7%)\n\n참고\n투자의견: Buy(매수) 유지\n\n목표주가 산정: 2025년 EPS에 Target P/E 21배 적용(2027년 EPS 기준으로는 9배 적용과 동일한 컨셉)\n\n첨부 리포트: 하나증권, 2025-07-21자 현대건설(000720) 기업분석/Earnings Review\n\n'

In [14]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "이 기업의 리포트 요약해줘."
response = chain.invoke(question)
print(response)
"""현대건설(000720) 리포트 요약 – 2025년 7월 21일자 하나증권
핵심 결론
투자의견: Buy(매수) 유지, 12개월 목표주가 83,000원으로 상향(기존 51,000원 대비 +62.7%)

상향 배경: 해외 원전 수주·매출 기여 기대 반영한 밸류에이션(25년 EPS에 P/E 21배, 27년 EPS 관점으로는 P/E 9배 적용과 동일)

주요 변수: 원전 수주 성사, 원전 인력 캐파 확장, 주택·건축 부문 리스크에 따른 변동 가능

2Q25 잠정실적(컨센서스 부합)
매출 7.7조원(-10.4% YoY), 영업이익 2,170억원(+47.3% YoY)

부문 GPM: 현대건설 별도 토목 6.1%, 건축주택 5.5%(전분기 대비 +1.2%p 개선), 플랜트 -0.3%(중동 원가 상승 반영, 클레임 진행 중)

현대엔지니어링 GPM 6.9%(분기 내 뚜렷한 개선 없음)

주택 동향: 2Q25 착공 4,941세대(1.2조원), 상반기 입주 0.7만세대, 하반기 1.7만세대 계획

미분양: 총 3,953세대, 준공 후 미분양 1,400세대 수준

수주/잔고: 2Q25 수주 7.3조원, 수주잔고 94.7조원(국내 73.2조, 해외 21.6조)

3Q25 전망
매출 7.4조원(-10.7% YoY), 영업이익 2,353억원(OPM 3.2%, +105.8% YoY) 추정

GPM 가정: 현대건설 5.6%(건축주택 5.5%, 토목 8.0%, 플랜트 5.0%), 현대엔지니어링 7.2%

변수: 해외 현장 추가 비용 반영 가능성, 현대엔지니어링 교량 사고 비용 반영 가능성

원전 모멘텀(목표가 상향의 핵심 논리)
인력 캐파 기준 동시 수행 가능 프로젝트 4개로 판단

가정된 프로젝트: 신한울 3·4호기, 불가리아 7·8호기, 홀텍 SMR(미국 펠리세이드 2기), 유럽 대형원전(스웨덴 혹은 슬로베니아)

4개 프로젝트 동시 진행 시 연매출 약 2.8조원, 평균 GPM 14.6%, 연간 이익 약 4,072억원 추정(하나증권 가정)

주가 및 밸류에이션 스냅샷
현재주가(리포트 기준일): 70,900원(2025-07-18)

시가총액: 약 7.90조원(리포트 표기), 발행주식수 1억 1,135만주, 외국인지분율 24.92%

컨센서스(연간): 2025E 매출 30.4조, 영업이익 9,836억원, 순이익 7,827억원, EPS 5,288원; 2026E 매출 30.6조, 영업이익 1.23조, EPS 6,814원

세부 운영 지표 하이라이트
주택 착공: 2Q25 현대건설 4,941세대(1.2조), 현대엔지니어링 450세대

입주 계획: 현대건설 하반기 1.7만세대, 현대엔지니어링 하반기 0.6만세대

수주 흐름: 2Q25 연결 수주 7.3조, 하반기 해외 대형 수주 가시성에 따라 변동성 존재

리스크 요인
중동 플랜트 원가 상승에 따른 추가 비용 반영 가능성 및 클레임 결과 불확실성

현대엔지니어링 교량 사고 관련 비용 반영 가능성

주택·건축 부문 추가 리스크 발생 시 목표주가 하향 요인

투자 포인트 요약
국내 주택 마진 개선이 진행되는 가운데, 해외 플랜트 비용 이슈는 있으나 클레임 진행으로 일부 상쇄 기대

원전 수주 모멘텀 및 수행 캐파 가시성 확대 시 중장기 실적·밸류에이션 리레이팅 가능

(요약 출처: 하나증권, 2025-07-21자 현대건설 기업분석/Earnings Review 리포트)"""

대양전기공업(108380)에 대한 IBK투자증권의 리포트 요약은 다음과 같습니다:  

- **투자의견 및 목표주가**: 매수의견 유지, 목표주가 33,000원(현재가 26,100원).  
- **2분기 실적**: 매출액 563억 원(전년 대비 +12.8%), 영업이익 58억 원(+89.4%)으로 컨센서스 부합.  
- **주요 사업부 동향**:  
  - **센서 사업부**: 상반기 매출 284억 원으로 전년 동기 대비 65% 달성. 하반기 계절성으로 인해 전년대비 48% 성장 전망. 2026년부터 유럽 시장 진출 계획.  
  - **선박용 조명**: 조선업 호황으로 수주 증가 및 이익률 개선.  
  - **방산 사업**: 글로벌 및 국내 업황 개선으로 안정적 성장 지속 예상.  
- **재무 전망**:  
  - 2025년 예상 매출액 2,396억 원, 영업이익 250억 원, EPS 2,408원.  
  - 2026년 이후 매출액과 이익 지속 성장 전망(영업이익률 10% 이상 유지 예상).  
- **밸류에이션**: PER 10.8배, PBR 0.9배로 저평가 상태.  

보고서는 대양전기공업이 사상 최대 매출과 이익률을 달성할 수 있는 최적의 환경을 갖췄다고 평가하며, 특히 센서 사업부의 성장 가능성을 강조했습니다.  

(참고: 리포트 작성일자는 2025년 8월 25일로, 미래 시점의 데이터임을 유의해 주세요.)


'현대건설(000720) 리포트 요약 – 2025년 7월 21일자 하나증권\n핵심 결론\n투자의견: Buy(매수) 유지, 12개월 목표주가 83,000원으로 상향(기존 51,000원 대비 +62.7%)\n\n상향 배경: 해외 원전 수주·매출 기여 기대 반영한 밸류에이션(25년 EPS에 P/E 21배, 27년 EPS 관점으로는 P/E 9배 적용과 동일)\n\n주요 변수: 원전 수주 성사, 원전 인력 캐파 확장, 주택·건축 부문 리스크에 따른 변동 가능\n\n2Q25 잠정실적(컨센서스 부합)\n매출 7.7조원(-10.4% YoY), 영업이익 2,170억원(+47.3% YoY)\n\n부문 GPM: 현대건설 별도 토목 6.1%, 건축주택 5.5%(전분기 대비 +1.2%p 개선), 플랜트 -0.3%(중동 원가 상승 반영, 클레임 진행 중)\n\n현대엔지니어링 GPM 6.9%(분기 내 뚜렷한 개선 없음)\n\n주택 동향: 2Q25 착공 4,941세대(1.2조원), 상반기 입주 0.7만세대, 하반기 1.7만세대 계획\n\n미분양: 총 3,953세대, 준공 후 미분양 1,400세대 수준\n\n수주/잔고: 2Q25 수주 7.3조원, 수주잔고 94.7조원(국내 73.2조, 해외 21.6조)\n\n3Q25 전망\n매출 7.4조원(-10.7% YoY), 영업이익 2,353억원(OPM 3.2%, +105.8% YoY) 추정\n\nGPM 가정: 현대건설 5.6%(건축주택 5.5%, 토목 8.0%, 플랜트 5.0%), 현대엔지니어링 7.2%\n\n변수: 해외 현장 추가 비용 반영 가능성, 현대엔지니어링 교량 사고 비용 반영 가능성\n\n원전 모멘텀(목표가 상향의 핵심 논리)\n인력 캐파 기준 동시 수행 가능 프로젝트 4개로 판단\n\n가정된 프로젝트: 신한울 3·4호기, 불가리아 7·8호기, 홀텍 SMR(미국 펠리세이드 2기), 유럽 대형원전(스웨덴 혹은 슬로베니아)\n\n4개 프로젝트 동시 진행 시 연매출 약 2.8조원, 평균 GPM 14.6%, 연간 이익 약 4,072억원 추정(하나

In [15]:
question = "워렌 버핏이라면 이 주식을 매수 했을까?"
response = chain.invoke(question)
print(response)

제공된 컨텍스트에서는 워렌 버핏의 투자 철학이나 대양전기공업에 대한 그의 구체적인 의견을 언급하지 않습니다. 다만, IBK투자증권의 보고서에서는 대양전기공업의 센서 및 기타 사업부 성장, 조선 및 방산 사업의 호조, 그리고 목표주가 33,000원 유지 등의 긍정적인 분석을 제시하고 있습니다.  

워렌 버핏은 일반적으로 "이해할 수 있는 사업", "강한 경쟁 우위", "탁월한 경영진", "합리적인 가격" 등의 기준을 중시합니다. 대양전기공업이 이러한 조건을 충족하는지 여부는 추가적인 정보 없이 판단하기 어렵습니다. 따라서 **"워렌 버핏이라면 이 주식을 매수했을까?"에 대한 명확한 답변은 제공할 수 없습니다**.  

(참고: 현재가 26,100원, 목표주가 33,000원으로 제시된 점은 투자 참고 사항일 뿐입니다.)
